!wget https://raw.githubusercontent.com/mesolitica/malaysian-dataset/master/llm-benchmark/BM-pt3/BM-A-pt3

In [1]:
import os

# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
#mesolitica/malaysian-llama2-7b-32k-instructions-v2
tokenizer = AutoTokenizer.from_pretrained('mesolitica/mallam-5B-4096')
# model = AutoModelForCausalLM.from_pretrained(
#     'fpf-7b-instructions-16k-call-v2/checkpoint-9300', 
#     use_flash_attention_2 = True, 
#     torch_dtype = torch.float16,
#     device_map="cuda:0"
# )

model = AutoModelForCausalLM.from_pretrained(
    'mesolitica/mallam-5B-4096', 
    #use_flash_attention_2 = True, 
    torch_dtype = torch.float16,
    device_map="auto"
)

/home/cybertron/anaconda3/envs/sgpt/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████████████| 3/3 [03:05<00:00, 61.77s/it]


In [3]:
from tqdm import tqdm
import os
import random
import json
random.seed(404)
torch.manual_seed(404)

data_folder = '/media/cybertron/fa54fcb6-b5e1-492e-978a-6389519c168a/llm-benchmarks/BM-PT3/data/'
with open(data_folder + 'BM-A-pt3') as fopen:
    text = fopen.read()
print(text.split('no: '))

In [5]:

data_folder = '/media/cybertron/fa54fcb6-b5e1-492e-978a-6389519c168a/llm-benchmarks/sejarah-spm/data/'
with open(data_folder+ 'sejarah_spm') as fopen:
    text = fopen.read()
    
questions = []
for t in text.split('no: ')[1:]: # split by "no"
    t = t.strip()
    no = t.split('\n')[0] # "split by newline to get index'
    soalan = t.split('soalan:')[1].split('jawapan:')[0].strip()
    jawapan = t.split('jawapan: ')[1].split(',')[0].strip()
    data = {
        'no': no,
        'soalan': soalan,
        'jawapan': jawapan,
    }
    questions.append(data)

In [6]:
questions[0]

{'no': '1',
 'soalan': 'Apakah yang dimaksudkan dengan wilayah pengaruh?\nA. Kawasan tanah jajahan dan takluk raja\nB. Kawasan yang dinaungi oleh raja dan kerabatnya\nC. Kawasan yang rakyatnya menerima dan memperakui pemerintahan seseorang raja\nD. Kawasan yang rakyatnya menganggap raja sebagai tuhan atau wakil tuhan',
 'jawapan': 'C'}

In [7]:
arange = set(range(len(questions)))
print(arange)

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19}


In [8]:
def convert_prompt(row, answer = False): # convert question to prompt 
    if answer: # include answer in prompt
        prompt = f""" 
soalan: {row['soalan']}
jawapan: {row['jawapan']}
    """
    else: # do not include answer in prompt
        prompt = f"""
soalan: {row['soalan']}
jawapan:
    """
    return prompt.strip()

def parse_mistral_chat(messages):

    user_query = messages[-1]['content']

    users, assistants = [], []
    for q in messages[:-1]:
        if q['role'] == 'user':
            users.append(q['content'])
        elif q['role'] == 'assistant':
            assistants.append(q['content'])

    texts = ['<s>']
    for u, a in zip(users, assistants):
        texts.append(f'[INST] {u.strip()} [/INST]{a.strip()}</s> ')

    texts.append(f'[INST] {user_query.strip()} [/INST]')
    prompt = ''.join(texts).strip()
    return prompt

In [10]:
for i in tqdm(range(len(questions))):
    
    prompts = []
    prompts.append(convert_prompt(questions[i])) # convert the question to prompt
    prompt = '\n\n'.join(prompts)
    messages = [
        {'role': 'user', 'content': prompt}
    ]
    prompt = parse_mistral_chat(messages)
    # print(prompt)
    # break
    inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda') # get input ids 
    repeat = []
    for _ in range(5): # for each question repeat 5 times
        try:
            generate_kwargs = dict(
                inputs,
                max_new_tokens=50,
                top_p=0.95,
                top_k=50,
                temperature=0.9,
                do_sample=True,
                num_beams=1,
                repetition_penalty=1.05,
            )
            r = model.generate(**generate_kwargs)
            splitted = tokenizer.decode(r[0]).split('[/INST]')[1].strip().replace('</s>', '').replace('.', '').replace(',', '').strip().split()
            splitted = [t for t in splitted if len(t) == 1 and t in {'A', 'B', 'C', 'D'}]
            repeat.append(splitted[0])
    
        except Exception as e:
            pass
    
    questions[i]['output'] = repeat

  0%|                                                    | 0/20 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
  5%|██▏                                         | 1/20 [00:10<03:27, 10.93s/it]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
 10%|████▍                                       | 2/20 [00:20<02:57,  9.86s/it]Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
Setting `pad

In [11]:
def most_common(l):
    return max(set(l), key=l.count)

In [12]:
filtered = [q for q in questions if 'output' in q and len(q['output'])]
correct = 0
for q in filtered:
    correct += most_common(q['output']) == q['jawapan']
(correct / len(filtered)) * 100

35.0

In [14]:
from transformers import AutoTokenizer, AutoModelForCausalLM


prompt = '<s>nama saya'
inputs = tokenizer([prompt], return_tensors='pt', add_special_tokens=False).to('cuda')

generate_kwargs = dict(
    inputs,
    max_new_tokens=512,
    top_p=0.95,
    top_k=50,
    temperature=0.9,
    do_sample=True,
    num_beams=1,
    repetition_penalty=1.05,
)
r = model.generate(**generate_kwargs)

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.


In [18]:
r

tensor([[    1,  2868,   689,   270,   454,  3010, 22722, 14277,  1332,  2018,
           353,  3698,    81,   663,     2,     1,  2231,    49, 27831,   282,
          1836,  2038,    93,   280,  2231,    49,   507,  8509, 18520,  2231,
         23959,    87,    66,    73,   443,  8623,  2231,    55,   333, 10285,
            78,   394,   476,   527,    17,     2,     1,   332,   591,  4625,
          3905,  6932, 11846,  2082,  4758,  2549, 13391, 27102,   273, 18040,
         20848,    17,   315,    83,  2457, 18959,    34,  2734,  1878,   802,
         14738,  3332,   341, 15843, 19412,   802,  3225,  7287,  1515,    87,
            17,  3865,    18,    24,    54,    91,    21,    46,    23,    44,
            27,    84,    40,     2,     1,  2231,  5131,    23,    73,    22,
           538,    75,    22,    73,    22,    92,    20,  2231, 11818,  1413,
         19908,  2231, 30514,  2060,    66,  8111,  2231, 11818,  1413, 19908,
          2231,  2341,    90,   507,    93,  1150,  

In [19]:
splitted = tokenizer.decode(r[0])
print(splitted)

<s> nama saya muhammad azfar bin zulkarnain</s><s> @NaufalAntezem @NanaMustafa @twt_fotografi @Tiktokk__ Wow.</s><s> Klu dulu suka ikut trend sebab tengok ramai org tweet psl tudung. Tp skrg? Aku rasa dia nk kena rotan sbb dia yg https://t.co/5Sx2K4I8qE</s><s> @sy4f3ysh3f3y1 @AzminAli @Ayahanda_MD @AzminAli @drwanazizah Suruh dia jawab sendiri! @mijjemije tuduh org yg fitnah dia sebagai rasis, gila kuasa etc. https://t.co/ZgR2YWlNM0</s><s> @Lynn_ibrahim76 @AinaaAlFayed @adibahnoor Dengar cerita ni!</s><s> @AzzizzHadi @MKNJPM @PDRMsia tolong ambil tindakan segera kepada pemandu-pemandu Grabcar yang tak ada common sense. https://t.co/9X5uFUo6VW</s><s> @hazieqaazmn @dilaaaaasrdn @nurinshafiqaa @ImanAbdulRahim @InjangNation Nak mintak tolong rt utk rezeki anak dia. Thankssss in advance!</s><s> @BoongaLavender Saja nak bagitau. Korang jangan buat macam tu kat anak-anak nanti boleh jadi geng cicak</s><s> @_bloominglyyy Tungguuuu. Rindu hg juga</s><s> @misschelsea @fckingchelsea17 Mcm tak cay